In [55]:
# google colab setup 
from google.colab import drive
drive.mount('/content/drive')

basepath = '/content/drive/MyDrive/research/Infinite/'

import sys
sys.path.append(basepath)

! pip install optuna
! pip install pytorch-lightning
! pip install transformers
! pip install seqeval

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning import seed_everything, loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint

from transformers import  DistilBertModel,DistilBertTokenizerFast

import random
import pandas as pd
import re

#from scripts.dataset import *
from scripts.utils import *
from scripts.model import IC_NER
#from data.ATIS.test_template import *
#from scripts.genContrastiveSample import *
seed_everything(42, workers=True)

Global seed set to 42


42

In [61]:
# model parameter
config = {

'mc' : {
    'model_name' : 'distilbert-base-cased',
    'tokenizer' : 'distilbert-base-cased',
    'joint_loss_coef' : 0.5,
    'id_1': 0.3986226896162205,
    'id_2':0.24620174329575167,
    'sd':0.28096911585616136,
    'Ihs': 181
},

# training parameters
'tc' : {
    'lr' : 0.00005,
    'epoch' : 40,
    'batch_size' : 32,
    'weight_decay' : 0.003,
    'shuffle_data' : True,
    'num_worker' : 2
},

# data params

'dc' : {
    "train_dir": "/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/train/clean/train.tsv",
    "val_dir": "/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/dev/clean/dev.tsv",
    'intent_num' : 18,
    'slots_num' : 120,
    'max_len' : 56
},

# misc
'misc' : {
    'fix_seed' : False,
    'gpus' : -1,
    'param_dir' : '/content/drive/MyDrive/research/Infinite/bin/AT/MC/',
    'log_dir' : '/content/drive/MyDrive/research/Infinite/logs/AT/MC/',
    'precision' : 16
}
}

basepath = '/content/drive/MyDrive/research/Infinite/'

In [62]:
# loading slot index file
final_slots = pd.read_csv( basepath + 'data/ATIS/slots_list.tsv',sep=',',header=None,names=['SLOTS']).SLOTS.values.tolist()
idx2slots  = {idx:slots for idx,slots in enumerate(final_slots)}

# callback for pytorch lightning
checkpoint_callback = ModelCheckpoint(
    monitor='val_IC_NER_loss',
    dirpath= config['misc']['param_dir'],
    filename='jointBert_AT-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min',
)

tb_logger = pl_loggers.TensorBoardLogger(config['misc']['log_dir'])

with open( basepath + 'data/BG_Noise_Phrase.txt') as f:
    content = f.readlines()

phrase = [x.strip() for x in content]

In [63]:
def slot_F1(pred,target,id2slots):

    pred_list = pred.tolist()
    target_list = target.tolist()
    
    pred_slots , target_slots = [],[]
    

    for idx_st,t in enumerate(target_list):
        pred_sample,target_sample = [],[]
        for idx_wt,wt in enumerate(t):

            if wt != -100:
                try:
                    target_sample.append(id2slots[wt])
                    pred_sample.append(id2slots[pred_list[idx_st][idx_wt]])
                except:
                    a=1

        pred_slots.append(pred_sample)
        target_slots.append(target_sample)
    
    return f1_score( target_slots, pred_slots,mode='strict', scheme=IOB2)

In [64]:
class nluDataset(Dataset):
    def __init__(self, file_dir):

        self.data = pd.read_csv(file_dir, sep="\t")
    
    def process_text(self,text):
        text = text.replace(".", "")
        text = text.replace("'", "")
        text = " ".join(text.split())
        return text


    def __getitem__(self, index):

        text = str(self.data.TEXT[index])
        text = self.process_text(text)
        
        
        # intent
        intent_id = self.data.INTENT_ID[index]
        intent_label = self.data.INTENT[index]

        # label processing
        slot_label = self.data.SLOTS[index]
        slot_id = self.data.SLOTS_ID[index]

        return {

            "text": text,

            "intent_id": intent_id,
            "intent_label": intent_label,

            "slots_id": slot_id,
            "slots_label": slot_label,
        }

    def __len__(self):
        return len(self.data)

In [65]:
class lightningDL(pl.LightningDataModule):
    def __init__(
        self, trainDS, valDS, testDS, mode , batch_size):

        super().__init__()
        self.trainDS = trainDS
        self.valDS = valDS
        self.testDS = testDS
        
        self.mode = mode
        self.batch_size = batch_size

    def setup(self, stage: [str] = None):
        
        if self.mode == 'AT':
            self.collate_fn_train = AT_collate_train
            self.collate_fn_val = AT_collate_val

    def train_dataloader(self):
        return DataLoader(
            self.trainDS, batch_size=self.batch_size, collate_fn=self.collate_fn_train)

    def val_dataloader(self):
        return DataLoader(
            self.valDS, batch_size=self.batch_size,collate_fn=self.collate_fn_val
        )

    def test_dataloader(self):
        return DataLoader(
            self.testDS, batch_size=self.batch_size,collate_fn=self.collate_fn_val
        )

In [66]:
def merge_text_label(text,slot):

    data = []
    for i in range(len(text)):
        data.append([text[i] , slot[i]])
    return data

def twolists(l1, l2 , prob):
    
    spl = [0]*int((1000)*prob) + [1]*int(1000*(1-prob))
    final = []
    while len(l1) >0 and len(l2) > 0:
        if random.sample(spl,1)[0] == 0:
            final.append(l1.pop(0))
        else:
            final.append(l2.pop(0))
            
    if len(l1) == 0:
        final = final + l2
    else:
        final = final + l1

    text,slot = '',''
    for token in final:
        text += token[0] + ' '
        slot += token[1] + ' '
    return ' '.join(text.split()), ' '.join(slot.split())

def BG_Noise(text,intent,slot, prob):
    
    augINTENT, augSLOTS, augTEXT = [],[],[]
    
    for i in range(len(text)):
        
        bg_TEXT = random.sample(phrase,1)[0]
        bg_SLOTS = re.sub(' +', ' ', '0 '*len(bg_TEXT.split(' ')))
        
        noisyData = merge_text_label(bg_TEXT.split(' '),bg_SLOTS.split(' '))
        cleanData = merge_text_label(text[i].split(' '),slot[i].split(' '))
        
        augText , augSlots = twolists(noisyData,cleanData,prob)
        
        augINTENT.append(intent[i])
        augTEXT.append(augText)
        augSLOTS.append(augSlots)

    return augTEXT, augINTENT, augSLOTS

def get_carrier_phrase_length(annotations):
    CP_idx = []
    for idx,token in enumerate(annotations.split(' ')):
        CP_idx.append(idx)
    
    return CP_idx 

def carrier_aug(text,intent,slot,tau):
      
    augINTENT, augSLOTS, augTEXT,augID = [],[],[],[]
    
    for i in range(len(text)):
        
        CP_idx = get_carrier_phrase_length(slot[i])
        
        CP_length = len(CP_idx)
        
        if CP_length <= 2:
            augINTENT.append(intent[i])
            augTEXT.append(text[i])
            augSLOTS.append(slot[i])
        else:
            del_count = int(CP_length/2) if CP_length <= 5 else int(tau*CP_length)
        
            del_index = random.sample(CP_idx,del_count)

            TEXT = ' '.join([i for j, i in enumerate(text[i].split(' ')) if j not in del_index])
            SLOTS = ' '.join([i for j, i in enumerate(slot[i].split(' ')) if j not in del_index])
         
            augINTENT.append(intent[i])
            augTEXT.append(TEXT)
            augSLOTS.append(SLOTS)
               
    return augTEXT, augINTENT, augSLOTS

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

def processSlotLabel(word_ids, slot_ids):

    # replace None and repetition with -100

    word_ids = [-100 if word_id == None else word_id for word_id in word_ids]

    previous_word = -100

    for idx, wid in enumerate(word_ids):

        if wid == -100:
            continue

        if wid == previous_word:
            word_ids[idx] = -100

        previous_word = wid

    slot_ids = list(map(int, slot_ids.split(" ")))
    new_labels = [
        -100 if word_id == -100 else slot_ids[word_id] for word_id in word_ids
    ]

    return new_labels

def AT_collate_train(batch):
    
    text,intent_id,slot_id = [],[],[]
    
    for datapoint in batch:
        text.append(datapoint['text'])
        intent_id.append(datapoint['intent_id'])
        slot_id.append(datapoint['slots_id'])
    
    # augmentation
    NC_tau = random.sample([0.20,0.40,0.60],1)[0]
    NC_text,NC_intent,NC_slot = carrier_aug(text,intent_id,slot_id,NC_tau)
    
    BG_prob = random.sample([0.25,0.50,0.75],1)[0]
    BG_text,BG_intent,BG_slot = BG_Noise(text,intent_id,slot_id,BG_prob)
    
    text = text  + NC_text #+ BG_text #
    intent_id = intent_id   + NC_intent #+ BG_intent
    slot_id = slot_id  +  NC_slot #+ BG_slot #
    # tokenization
    
    token_ids , mask , slot_out = [],[],[]
    for i in range(len(text)):
        
        inputs = tokenizer.encode_plus(text[i],None,add_special_tokens=True,return_token_type_ids=False,
        truncation=True,max_length=56,padding="max_length")
        word_ids = inputs.word_ids()
        slot_out.append(processSlotLabel(word_ids, slot_id[i]))
        
        token_ids.append(inputs["input_ids"])
        mask.append(inputs["attention_mask"])
    
    # slot_processing
    
    
    token_ids = torch.tensor(token_ids, dtype=torch.long)
    mask = torch.tensor(mask, dtype=torch.long)
    intent_id = torch.tensor(intent_id, dtype=torch.long)
    slot_out = torch.tensor(slot_out, dtype=torch.long)
    
    
    return {'token_ids':token_ids , 'mask':mask , 'intent_id':intent_id,'slots_id':slot_out}

def AT_collate_val(batch):
    
    text,intent_id,slot_id = [],[],[]
    
    for datapoint in batch:
        text.append(datapoint['text'])
        intent_id.append(datapoint['intent_id'])
        slot_id.append(datapoint['slots_id'])
    
    # tokenization
    
    token_ids , mask , slot_out = [],[],[]
    for i in range(len(text)):
        
        inputs = tokenizer.encode_plus(text[i],None,add_special_tokens=True,return_token_type_ids=False,
        truncation=True,max_length=56,padding="max_length")
        word_ids = inputs.word_ids()
        slot_out.append(processSlotLabel(word_ids, slot_id[i]))
        
        token_ids.append(inputs["input_ids"])
        mask.append(inputs["attention_mask"])
    
    # slot_processing
    
    
    token_ids = torch.tensor(token_ids, dtype=torch.long)
    mask = torch.tensor(mask, dtype=torch.long)
    intent_id = torch.tensor(intent_id, dtype=torch.long)
    slot_out = torch.tensor(slot_out, dtype=torch.long)
    
    
    return {'token_ids':token_ids , 'mask':mask , 'intent_id':intent_id,'slots_id':slot_out}

In [67]:
class jointBert_AT(pl.LightningModule):
    
    def __init__(self, cfg):
        super().__init__()
        
        self.IC_NER = IC_NER(cfg)
        self.cfg = cfg

    def forward(self, input_ids, attention_mask , intent_target, slots_target):
        return self.IC_NER(input_ids, attention_mask , intent_target, slots_target)

    def training_step(self, batch, batch_idx):
        
        token_ids, attention_mask = batch['token_ids'], batch['mask']
        intent_target,slots_target = batch['intent_id'], batch['slots_id']
        
        out = self(token_ids,attention_mask,intent_target,slots_target)
        
        self.log('train_IC_NER_loss', out['joint_loss'], on_step=False, on_epoch=True, logger=True)
        self.log('train_IC_loss', out['ic_loss'], on_step=False, on_epoch=True, logger=True)
        self.log('train_NER_loss', out['ner_loss'], on_step=False, on_epoch=True, logger=True)
        
        return out['joint_loss']
    
    def validation_step(self, batch, batch_idx):
        
        token_ids, attention_mask = batch['token_ids'], batch['mask']
        intent_target,slots_target = batch['intent_id'], batch['slots_id']
        
        out = self(token_ids,attention_mask,intent_target,slots_target)
        intent_pred, slot_pred = out['intent_pred'], out['slot_pred']
        
        self.log('val_IC_NER_loss', out['joint_loss'], on_step=False, on_epoch=True,  logger=True)
        self.log('val_IC_loss', out['ic_loss'], on_step=False, on_epoch=True,  logger=True)
        self.log('val_NER_loss', out['ner_loss'], on_step=False, on_epoch=True,  logger=True)
        self.log('val_intent_acc', accuracy(out['intent_pred'],intent_target), on_step=False, on_epoch=True,  logger=True)
        self.log('slot_f1', slot_F1(out['slot_pred'],slots_target,idx2slots), on_step=False, on_epoch=True, logger=True)
        
        
        return out['joint_loss']
    
    def test_step(self,batch,batch_idx):
        
        token_ids, attention_mask = batch['token_ids'], batch['mask']
        intent_target,slots_target = batch['intent_id'], batch['slots_id']
        
        out = self(token_ids,attention_mask,intent_target,slots_target)
        intent_pred, slot_pred = out['intent_pred'], out['slot_pred']
        
        self.log('test_intent_acc', accuracy(intent_pred,intent_target), on_step=False, on_epoch=True,  logger=True)
        self.log('test_slot_f1', slot_F1(slot_pred,slots_target,idx2slots), on_step=False, on_epoch=True, logger=True)
        
        return out['joint_loss']
        

    def configure_optimizers(self):
         return torch.optim.AdamW([
                {'params': self.IC_NER.encoder.parameters() , 'lr' : 0.00005 , 'weight_decay': config["tc"]["weight_decay"]},
                {'params': self.IC_NER.intent_FC1.parameters(), 'lr': 0.00048299051416981716},
                {'params': self.IC_NER.intent_FC2.parameters(), 'lr': 0.00048299051416981716},
                {'params': self.IC_NER.slots_FC.parameters(), 'lr': 0.0008757239486275598}])

In [68]:
# initialize the dataloader

trainDS = nluDataset( '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/train/clean/train.tsv')
valDS = nluDataset( '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/dev/clean/dev.tsv')

lightningDataLoader = lightningDL(trainDS,valDS,valDS,"AT", config['tc']['batch_size'])

# initialize the model
model = jointBert_AT(config)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [69]:
# model training
trainer = pl.Trainer(gpus=config['misc']['gpus'], deterministic=True, logger=tb_logger, callbacks=[checkpoint_callback] ,precision=config['misc']['precision'],max_epochs=config['tc']['epoch'], check_val_every_n_epoch=1)

trainer.fit(model, lightningDataLoader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params
----------------------------------
0 | IC_NER | IC_NER | 65.4 M
----------------------------------
65.4 M    Trainable params
0         Non-trainable params
65.4 M    Total params
261.703   Total estimated model params size (MB)


Global seed set to 42


In [70]:
test_50BG = [ '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseBG/50per/test_01.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseBG/50per/test_02.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseBG/50per/test_04.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseBG/50per/test_05.tsv']

test_25BG = [ '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseBG/25per/test_01.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseBG/25per/test_02.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseBG/25per/test_04.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseBG/25per/test_05.tsv']

test_75BG = [ '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseBG/75per/test_01.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseBG/75per/test_02.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseBG/75per/test_03.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseBG/75per/test_04.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseBG/75per/test_05.tsv']

test_20OOC = [ '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/20per/test_01.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/20per/test_02.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/20per/test_03.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/20per/test_04.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/20per/test_05.tsv']

test_40OOC = [ '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/40per/test_01.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/40per/test_02.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/40per/test_03.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/40per/test_04.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/40per/test_05.tsv']


test_60OOC = [ '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/60per/test_01.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/60per/test_02.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/60per/test_03.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/60per/test_04.tsv',
              '/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/NoiseOOC/60per/test_05.tsv']

test_clean = ['/content/drive/MyDrive/research/Infinite/data/ATIS/experiment/test/clean/test.tsv']

In [71]:
def cal_mean_stderror(metric):
    
    if len(metric) == 1:
        return metric
    var,std_error = 0,0
    mean = sum(metric)/len(metric)
    for m in metric:
        var += (m-mean)**2
    var = (var/(len(metric)-1))**0.5
    std_error = var/((len(metric))**0.5)
    return [round(mean,4),round(std_error,4)]

In [79]:
# testing the model

acc,slotF1 = [],[]

              
model_path = ['/content/drive/MyDrive/research/Infinite/bin/AT/MC/jointBert_AT-epoch=11-val_loss=0.00.ckpt',]


for m in model_path:
    model = jointBert_AT.load_from_checkpoint(m,cfg=config)
    
    for test_fn in test_25BG:
        testDS = nluDataset(test_fn)
        lightningDataLoader = lightningDL(trainDS,valDS,testDS,"AT", config['tc']['batch_size'])
    
        lightningDataLoader.setup() 
        test = lightningDataLoader.test_dataloader()
        out = trainer.test(model=model ,test_dataloaders=test)
        acc.append(out[0]['test_intent_acc'])
        slotF1.append(out[0]['test_slot_f1'])
    
print('acc:',cal_mean_stderror(acc),'slotsF1',cal_mean_stderror(slotF1))

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_intent_acc': 0.9384098649024963, 'test_slot_f1': 0.7805275321006775}
--------------------------------------------------------------------------------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_intent_acc': 0.9227323532104492, 'test_slot_f1': 0.7685649394989014}
--------------------------------------------------------------------------------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_intent_acc': 0.9227323532104492, 'test_slot_f1': 0.7708908319473267}
--------------------------------------------------------------------------------



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_intent_acc': 0.9339305758476257, 'test_slot_f1': 0.7758455276489258}
--------------------------------------------------------------------------------
acc: [0.9295, 0.004] slotsF1 [0.774, 0.0027]
